In [7]:
!pip install torch


   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 2.8 MB/s eta 0:01:12
   ---------------------------------------- 1.0/203.1 MB 3.4 MB/s eta 0:01:01
   ---------------------------------------- 2.1/203.1 MB 3.5 MB/s eta 0:00:59
    --------------------------------------- 2.6/203.1 MB 3.3 MB/s eta 0:01:02
    --------------------------------------- 3.4/203.1 MB 3.5 MB/s eta 0:00:58
    --------------------------------------- 4.2/203.1 MB 3.6 MB/s eta 0:00:56
   - -------------------------------------- 5.2/203.1 MB 3.7 MB/s eta 0:00:54
   - -------------------------------------- 6.0/203.1 MB 3.7 MB/s eta 0:00:53
   - -------------------------------------- 6.6/203.1 MB 3.8 MB/s eta 0:00:53
   - -------------------------------------- 7.3/203.1 MB 3.6 MB/s eta 0:00:55
   - -------------------------------------- 7.9/203.1 MB 3.5 MB/s eta 0:00:57
   - -------------------------------------- 8.4/203.1 MB 3.5 MB/s eta 0

In [15]:
import pandas as pd
import torch
import nltk
import spacy
from transformers import BertTokenizer, BertModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
df = pd.read_csv("D:\\epita class notes\\semester - 3\\action learnign\\project repository\\Hate_speech_detection_using_data_augmentation\\Hate_speech_detection_using_data_augmentation\\data\\cleaned_dataset\\labeled_data_cleaned.csv")
df

,corrected_tweet,class
0,as a woman you shouldn't complain about cleani...,2
1,boy date coldtyga own bad for coffin dat hoe i...,1
2,dawn you ever fuck a bitch and she start to cr...,1
3,she look like a granny,1
4,they shit you hear about me might be true or i...,1
...,...,...
995,munda out here sucking bitches howdhow,1
996,bitch if your they hobbit you need to let me k...,1
997,these folks so bad in just here to talk trash,1
998,brittany bitch a my dog man,1


In [19]:
lemmatizer = WordNetLemmatizer()

In [20]:
def preprocessing_text(text):
    words = word_tokenize(text)  
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word, pos=wordnet.VERB) if wordnet.synsets(word) else lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [21]:
df['preprocessed_tweet'] = df['corrected_tweet'].apply(preprocessing_text)
print(df[['corrected_tweet', 'preprocessed_tweet', 'class']])

                                       corrected_tweet  \
0    as a woman you shouldn't complain about cleani...   
1    boy date coldtyga own bad for coffin dat hoe i...   
2    dawn you ever fuck a bitch and she start to cr...   
3                               she look like a granny   
4    they shit you hear about me might be true or i...   
..                                                 ...   
995             munda out here sucking bitches howdhow   
996  bitch if your they hobbit you need to let me k...   
997      these folks so bad in just here to talk trash   
998                        brittany bitch a my dog man   
999                          sch bitch selling her ass   

                                    preprocessed_tweet  class  
0    woman n't complain clean house amp man always ...      2  
1        boy date coldtyga bad coffin dat hoe st place      1  
2          dawn ever fuck bitch start cry confuse shit      1  
3                                     look like

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


D:\miniconda\miniconda\envs\action_learning\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

D:\miniconda\miniconda\envs\action_learning\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\edwin victor\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [23]:
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embeddings

In [25]:
lemmatized_text = df['preprocessed_tweet'].iloc[0] 

In [26]:
embedding = get_bert_embeddings(lemmatized_text)

print("Shape of BERT embedding:", embedding.shape)

Shape of BERT embedding: (768,)


In [28]:
embedding


array([-7.51441568e-02,  4.98067468e-01, -5.56070246e-02, -5.44923358e-03,
       -5.27820230e-01, -1.13163456e-01,  4.41746265e-01,  2.92903781e-01,
        7.97836930e-02,  1.28294632e-01,  1.69782355e-01,  1.51524037e-01,
       -2.60809839e-01,  3.46482992e-01, -1.80287268e-02,  5.94214052e-02,
        4.21929002e-01,  2.41342306e-01,  4.35425788e-01, -1.92416478e-02,
       -2.07862519e-02, -4.99410741e-02,  2.98909158e-01, -1.30977422e-01,
        5.99727929e-02, -8.97464529e-02, -1.33891046e-01,  8.75546262e-02,
        5.03197536e-02,  3.09042543e-01,  1.81334138e-01,  5.06177425e-01,
       -2.32073009e-01, -4.51614946e-01,  1.97397500e-01, -2.85742342e-01,
        2.35904962e-01, -1.99015439e-02,  3.76851857e-01,  9.66946334e-02,
        1.44756630e-01, -3.70326713e-02,  2.95733631e-01, -4.96355817e-02,
       -6.94790334e-02, -1.88755900e-01, -3.05208802e+00,  1.87981471e-01,
       -1.75687343e-01, -4.90011960e-01,  5.71520030e-01, -4.93127227e-01,
        1.69535935e-01,  